In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import sqlite3    
import re   
import requests   
from bs4 import BeautifulSoup   
import plotly.graph_objects as go
#import win32com.client as win32
import plotly.io as pio
from plotly.subplots import make_subplots

In [2]:
pd.set_option('display.max_rows', None)

#### Funções Data ( não são usadas para as funções SQL)

In [3]:
# Dado o offset um número inteiro, retorna a data de agora menos o offset.

def day_n (offset):         
    
    return datetime.today()-timedelta(days = offset)         

In [4]:
def sunday(offset, day):    
    
    if day.weekday()==6:     
        day = day_n(offset+2)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)         
    
    return day   

####  Caso seja domingo essa função retorna o última dia antes de começar o final de semana
####  seja ele feriado ou não.  
####  Caso não seja domingo ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [5]:
def saturday(offset, day):
    
    if day.weekday()==5:     
        day = day_n(offset+1)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)     
    return day   

####  Caso seja sabado essa função retorna o última dia antes de começar o final de semana, 
####  seja ele feriado ou não.  
####  Caso não seja sábado ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [6]:
def holiday(offset, day, dataf):     
    
    for i in dataf:     
        if (i[8:]+"-"+i[5:7]+"-"+i[0:4])==day.strftime("%d-%m-%Y"):             
            return holiday(offset+1, day_n(offset+1), dataf)         
    
            
    if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):         
    
        
        if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):             
            
            return day         
        else:             
            return saturday(offset, day)         
    else:         
        return sunday(offset, day)         

In [7]:
##### Descobrindo o D-1 #####

def find_last_workday( offset, day, dataf): 
    
    if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
        
        if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):             
            
            if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):                 
                return day                 
                
            else:                     
                return saturday(offset, day)         
        else:                 
            return sunday(offset, day)         
    else:         
        return holiday(offset, day, dataf)          

#### Vetor de feriados ( não são usadas para as funções SQL)

In [8]:
def holidays():
    
    feriados = pd.read_csv(r"C:\Users\psilv\Python\feriados.csv")   
 
 
    data = feriados['data'].tolist()   
 
    ########### Cria vetor de feriados 
    
    dataf=[] 
    
    for i in data:   
        
        dataf.append(i.strip()[-4:] + "-" + ("0"+ i.strip()[0] if i.find("/")==1 else i.strip()[0:i.find("/")]) + "-" + ("0" + re.search("/(.*?)/", i.strip()).group(1) if len(re.search("/(.*?)/", i.strip()).group(1))==1 else re.search("/(.*?)/", i.strip()).group(1)))
        
    return dataf     

#### Vetor de feriados ( é usada para as funções SQL)

In [9]:
def past_n_days(n):     
    day = datetime.today()-timedelta(days = n)     
    print("O último dia corrido é: " +  day.strftime("%d-%m-%Y"))     
    return day   

####  Função auxiliar para criar Vetor de dias úteis ( é usada para as funções SQL)

In [10]:
##### Date Format input  YY-MM-dd,  run_days são números inteiros

##### O input de data dentro da função pandas.date_range deve ser no formato YY-MM-dd

def date_array (last_day, run_days):

   
    Datas = pd.date_range(end = last_day.strftime("%Y-%m-%d"), periods = run_days).to_pydatetime().tolist()

    
    datas=[] 

    
    for k in range(len(Datas)):

        datas.append(Datas[len(Datas) - k -1])
 
    return datas

#### Função iterativa para criar Vetor de dias úteis. ( não são usadas para as funções SQL)

In [12]:
#### Função para obter um vetor de dias úteis de um tamanho específico, por ser iterativa é custosa computacionalmente

#### Date format input dd-MM-YY, length é número inteiro


def work_days_array_length(last_day, length):


    i=length

    while len(date_array(last_day, i))!= length:

        if len(date_array(last_day, i))==length:

            break

        i+=1

    return date_array(last_day, i)

#### Função  Vetor de dias úteis. ( é usada para as funções SQL)

In [75]:
####### Criando vetor desde o ultimo dia do banco até o D-1

####### Start é a mais antiga, e End a mais recente

####### Date Format input  YY-MM-dd (SQL)


def  work_days_array_btw_dates(start_date, end_date):
    

    end = end_date
    start = start_date

    
    dif = end-start
    dif = int(abs(dif.days))
    
    
    print("Números de dias a serem atualizados:  " + str(dif))
    print("\n")
    
    #### No caso start_date=end_date é preciso desse if 
    
    if dif == 0:
        dif=2
            

    dates = date_array(end_date, 2*dif)

    aux=0
    

    for i in dates:

        if i==start_date:

            aux = dates.index(i)
                 
    
    return dates[:aux+1]

#### Função para deletar linha específica

In [16]:
def delete_row_sql(table_name, column_name, conn, desired_info):
    
    
    query = ("""DELETE""" + 
             """ FROM """ + table_name +
             """ WHERE """ + column_name +
             """ = '""" + desired_info +
             """'; """)
    
    conn.execute(query)
    
    print("Deleted as Follows:")
    print("\n")
    print(query)

#### Função para obter um dataframe a partir de uma query do banco de dados

In [17]:
### Criar função para obter um dataframe entre datas específicas a partir de uma query em SQL

def df_from_sql_query(table_name, date_column_name, conn, start_date, end_date):
    
    
    query = ("""SELECT * """ + 
             """ FROM """ + table_name +
             """ WHERE """ + date_column_name +
             """ BETWEEN strftime('%Y-%m-%d %H:%M:%S', '""" + start_date +
             """') AND strftime('%Y-%m-%d %H:%M:%S', '""" + end_date +
             """') ORDER BY """ + date_column_name + """; """)
    
    #print(query)

    df = pd.read_sql_query(query, conn)
    
    print(df.dtypes)
    
    
    return df   

#### Inserindo novas linhas no banco de dados

In [19]:
##### Tornar essa função genérica para qualquer dataframe

def insert_n_lines_database(cursor, df, table_name, date_column_name):
    
           
    print("\n")     
    print("Os dados antes da atualização eram:")   
    print("\n")  
    
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)             
 
    print("\n")     
   
    insert_n_rows(cursor, df, table_name)             
       
    print("\n")   
    print("Os novos dados são:")   
    print("\n")     
                   
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)         
    print("\n")             
            

#### Atualizando as bases de dados

In [20]:
def updating_database(cursor, date_column_name):      
    
    for i in assets:
        
        print('\n')
        
        day = past_n_days(1)
    
        #day = day.strftime("%Y-%m-%d")
        
        ###### Verificando a última data atualizada no banco de dados   
        last_date_db = get_last_db_date(cursor, date_column_name, i)
        
        

        if last_date_db.strftime("%Y-%m-%d") == day.strftime("%Y-%m-%d"):         
            print("Os dados da cripto " + str(i) + " estão Atualizados!!")         
        else:         
            print("\n")     
            print("O último dia atualizado no banco de dados é: " + last_date_db.strftime("%Y-%m-%d"))   
            print("\n")         

            #### Vetor de datas   
            dates = work_days_array_btw_dates(last_date_db, day)[:-1]   
            print("Datas a serem atualizadas no banco de dados:")         
            print("\n")         

            for j in range(len(dates)):             
                print(dates[j])             


            print("\n")
            print("Downloading from Yahoo Finance API ....")         
            print("\n")         

            #### Dataframe com os resultados
            
            print("Moeda Atualizada: " + assets[i])
            
            print(last_date_db.date())
            print(day.date())
            print((day-last_date_db).days)
            
            print(dates)
            print(dates[0])
            print(dates[0].strftime("%Y-%m-%d"))
            print(dates[-1:][0].strftime("%Y-%m-%d"))
            
    

            df = get_data_btw_date(assets[i],dates[-1:][0].strftime("%Y-%m-%d"), dates[0].strftime("%Y-%m-%d"))     


            #### Inserindo novos dados           
            insert_n_lines_database(cursor, df, i, date_column_name)  

            ##### Adicionar o insert lines dentro da função Web Scrap para atualizar o banco de dados em cada iteração e não
            ##### correr o risco de falhar o web scraping e perder os dados
    
    

#### Função para captar os dados da API do Yahoo Finance

In [21]:
#### Nome da moeda, exemplo: Ethereum é ETH-USD
#### Data format YY-MM-DD

def get_data_btw_date(crypto_name, start_date, end_date):
    
    crypto_df = yf.download(crypto_name, start= start_date, end= end_date)
    
    crypto_df = crypto_df.reset_index(level=0)
    
    #print(crypto_df.dtypes)
    #print(crypto_df)
    
    ## Agora o banco de dados recebe a coluna de data como tipo data, ateriormente era texto. 
    ##As operações abaixo transformavam para texto.
    
    #crypto_df['Date'] = pd.to_datetime(crypto_df['Date'], format='%Y-%m-%d %H:%M:%S')
    
    #crypto_df['Date'] = crypto_df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    #print(crypto_df.dtypes)
    #print(crypto_df)

    return crypto_df

#### Funções SQL

In [22]:
#### index can be False or True, to me worked as False

def create_table(df, name_desired, conn, index):
    
        df.to_sql(name_desired, conn, if_exists='fail', index=index)
        

In [24]:
def rename_table(table_name, new_table_name, cursor):
    
    try:
        
        for i in get_names_tables(cursor):
        
            if new_table_name == i[0]:
                
                print("O Nome já está em uso!")
            
                raise Exception

        cursor.execute("""ALTER TABLE """ +  table_name + """ RENAME TO """ + new_table_name + """ ;""")

    except Exception:

        print("Não foi possível alterar o nome da tabela!")
        

In [25]:
def show_table(table_name, cursor):     
    
    c=cursor.execute("""SELECT * FROM """ + table_name ).fetchall()     
    for i in c:      
        print(i)     
    return c     

In [26]:
def delete_table(name, cursor):   
    
    cursor.execute("""DROP TABLE """ +  name ).fetchall()   

In [27]:
def last_database_day(crypto_name):

    sql_str = '''SELECT Date FROM ''' + crypto_name + ''' ORDER BY Date DESC LIMIT 1;'''

    c.execute(sql_str)

    date = c.fetchone()

    return date[0][:10]
    

In [28]:
def get_names_tables(cursor):     
    
    cursor.execute('''SELECT name FROM sqlite_master WHERE type='table';''')   
    
    return(cursor.fetchall())

In [29]:
def count_rows_table(cursor, table_name): 
    
    c = cursor.execute("""SELECT COUNT(*) FROM """ + table_name).fetchall()     
    return c[0][0]   

In [30]:
def get_columns_names(cursor, table_name): 
    
    c = cursor.execute("""PRAGMA table_info(""" + table_name + """);  """ ).fetchall()     
    col_names = []     
    for i in c:         
            col_names.append(i[1])
            
    return col_names   

In [31]:
def get_last_n_rows(cursor, table_name, n_rows, column_name):
    
    c = cursor.execute("""SELECT * FROM """ 
                       + table_name + 
                       """ ORDER BY """ +column_name + 
                       """ DESC LIMIT """+ str(n_rows) + 
                       """ ; """  ).fetchall()     
#     for i in c:      
#         print(i)


    
    return c     

In [32]:
def insert_n_rows(cursor, df, table_name):     
 
   
    new_values = []   
    
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')
    
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    for i in range(len(df)):
        
        
        new_values.append(tuple(df.iloc[i]))
        
    
    s=""     
    
    for i in new_values:   
        
        s+=str(i) + ","   
        
    
    s=s[:-1] 

    print("")
    
    
    cursor.execute("""INSERT INTO """ + table_name + """ VALUES """ + s + """ ; """  )     
    

In [33]:
def distinct_values(table_name, column_name, cursor):
    
    c = cursor.execute("""SELECT """ + column_name + """, COUNT(*) """ + """ FROM """ + table_name + """ GROUP BY """ +  column_name + """ ; """  ).fetchall()     
    
    
    for i in c:
        
        print(i)
    
       

In [34]:
def change_columns_names(cursor, table_name, column_old_name, column_new_name):   
 
    print("Old Columns Names: " + str(get_columns_names(cursor, table_name)) )     
    c = cursor.execute("""ALTER TABLE """ + table_name +                                 """ RENAME COLUMN """ + column_old_name +                                 """ TO """+column_new_name+                                 """ ; """  ).fetchall()     
    print("New Columns Names: " + str(get_columns_names(cursor, table_name)) )     
 

In [35]:
def get_last_db_date(cursor,column_name, table_name):  
    
    c=cursor.execute("""SELECT """ + column_name + 
                     """ FROM """ + table_name + 
                     """ ORDER BY """ + column_name + 
                     """ DESC LIMIT 1 ; """).fetchall()     
    return c[0][0]   

In [37]:
def database_info(cursor, table_name, date_column_name):
    
        
    
    print("O banco de dados contém as seguintes tabelas: ")
    
    print("\n")
    
    for i in get_names_tables(cursor):
        print(i[0])
    
    print("\n")
    
    print("O nome da tabela solicitada é :" + table_name)
    
    print("\n")
    
    print("O nome das colunas são: ")
    
    print("\n")
    
    print(get_columns_names(cursor, table_name))
    
    print("\n")
    
    print("O número total de linhas é: " + str(count_rows_table(cursor, table_name)))
    
    print("\n")
    
    print("A data mais recente dos dados é: " + str(get_last_db_date(cursor,date_column_name, table_name)))
    
    print("\n")
    
    print("Todas as datas distintas e quantas linhas cada uma possui são: ")
    
    print("\n")
    
    print(distinct_values(table_name, date_column_name, cursor)) 
    
    

In [38]:
def get_new_crypto_in_yf(crypto_name, table_title, conn, end_date, index=False):
    
 
    df = get_data_btw_date(crypto_name, '2000-01-01', end_date)
        
    #### Data escolhida é bem antes do surgimento das criptomoedas no planeta terra
        
        
    create_table(df, table_title, conn, index)
    

In [39]:
###### updating entire database for all assets ##########

def insert_new_criptos_in_db(assets):
    
    day = past_n_days(1)
    
    day = day.strftime("%Y-%m-%d")

    for i in assets:
        
        try:

            get_new_crypto_in_yf(assets[i], i, conn, day, index=False)
            
        
        except Exception:
            
            print(str(assets[i]+ " já  está no banco de dados!"))
            
            continue

In [40]:
assets = { 'BTC': 'BTC-USD', 
           'ETH': 'ETH-USD', 
           'USDT':'USDT-USD', 
           'USDC':'USDC-USD', 
           'BNB':'BNB-USD',
           'BUSD':'BUSD-USD',
           'XRP': 'XRP-USD',
           'ADA': 'ADA-USD',
           'SOL': 'SOL-USD',
           'DOGE':'DOGE-USD',
           'DOT': 'DOT-USD',
           'HEX': 'HEX-USD',
           'SHIB': 'SHIB-USD',
           'DAI': 'DAI-USD',
           'WTRX': 'WTRX-USD',
           'AVAX':'AVAX-USD',
           'MATIC': 'MATIC-USD',
           'TRX': 'TRX-USD',
           'STETH': 'STETH-USD',
           'WBTC': 'WBTC-USD',
           'LEO': 'LEO-USD',
           'ETC': 'ETC-USD',
           'LTC': 'LTC-USD'
         }

#### Rodando

In [72]:
## These detect_types allow us to read date columns as date type that was stored as a string by sqlite3

conn = sqlite3.connect('crypto_data', detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)
c = conn.cursor()


In [74]:
conn.commit()
c.close()

In [ ]:
# Datetime tips

# datetime(2023,6,13)

# datetime(2023,6,13,0,0,0)

# datetime(2023,6,13).date()

# date_array(datetime.today(),5)

In [ ]:
#df_from_sql_query('ETH', "Date", conn, "2022-01-28", "2022-07-25")

In [ ]:
#a = get_data_btw_date('BTC-USD', '2023-01-12', '2023-06-05')

In [66]:
#get_last_db_date(c,"Date", "ADA")

In [73]:
updating_database(c, "Date")



O último dia corrido é: 13-06-2023
Os dados da cripto BTC estão Atualizados!!


O último dia corrido é: 13-06-2023


O último dia atualizado no banco de dados é: 2023-06-12


2023-06-12 00:00:00
Números de dias a serem atualizados:  1


[datetime.datetime(2023, 6, 13, 0, 0), datetime.datetime(2023, 6, 12, 0, 0)]
2023-06-12 00:00:00
Datas a serem atualizadas no banco de dados:


2023-06-13 00:00:00




Moeda Atualizada: ETH-USD
2023-06-12
2023-06-13
1
[datetime.datetime(2023, 6, 13, 0, 0)]
2023-06-13 00:00:00
2023-06-13
2023-06-13
[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 1753.075439453125, 1757.6597900390625, 1722.9063720703125, 1742.5286865234375, 1742.5286865234375, 6031384958)
(datetime.datetime(2023, 6, 11, 0, 0), 1752.527587890625, 1776.84716796875, 1741.1104736328125, 1753.415283203125, 1753.415283203125, 4559112981)
(datetime.datetime(2023, 6, 10, 0, 0), 1840.3919677734375

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 0.9996110200881958, 1.000277042388916, 0.9988330006599426, 0.9997580051422119, 0.9997580051422119, 2234723372)
(datetime.datetime(2023, 6, 11, 0, 0), 0.9996529817581177, 1.0001989603042603, 0.9993919730186462, 0.9997640252113342, 0.9997640252113342, 1535457755)
(datetime.datetime(2023, 6, 10, 0, 0), 0.999966025352478, 1.0005509853363037, 0.9972730278968811, 0.9996150135993958, 0.9996150135993958, 3169217883)
(datetime.datetime(2023, 6, 9, 0, 0), 0.9999150037765503, 1.0004090070724487, 0.9992660284042358, 0.9999679923057556, 0.9999679923057556, 1842949247)
(datetime.datetime(2023, 6, 8, 0, 0), 0.9998760223388672, 1.0002919435501099, 0.9993759989738464, 0.9998850226402283, 0.9998850226402283, 1884894804)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 0.9997439980506896, 1.0006959438323977, 0.9989449977874756, 0.999813973903

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 0.06157900020480156, 0.06163499876856804, 0.06040399894118309, 0.061459001153707504, 0.061459001153707504, 210832319)
(datetime.datetime(2023, 6, 11, 0, 0), 0.06186100095510483, 0.06251499801874161, 0.06130300089716911, 0.06158699840307236, 0.06158699840307236, 235910036)
(datetime.datetime(2023, 6, 10, 0, 0), 0.0694890022277832, 0.07059100270271301, 0.05853600054979324, 0.06185999885201454, 0.06185999885201454, 975736541)
(datetime.datetime(2023, 6, 9, 0, 0), 0.06796299666166306, 0.07091999799013138, 0.0676259994506836, 0.0694890022277832, 0.0694890022277832, 274101529)
(datetime.datetime(2023, 6, 8, 0, 0), 0.06730200350284576, 0.06842599809169769, 0.06715399771928787, 0.06796599924564362, 0.06796599924564362, 167825308)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 0.061459001153707504, 0.06245300173759461, 0.060763999

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 0.9995560050010681, 1.000967025756836, 0.9970189929008484, 0.9997119903564453, 0.9997119903564453, 128701031)
(datetime.datetime(2023, 6, 11, 0, 0), 0.9998379945755005, 1.000920057296753, 0.9981989860534668, 0.9996359944343567, 0.9996359944343567, 87455639)
(datetime.datetime(2023, 6, 10, 0, 0), 0.9997410178184509, 1.0015300512313843, 0.9954590201377869, 0.9999409914016724, 0.9999409914016724, 235160228)
(datetime.datetime(2023, 6, 9, 0, 0), 0.9995959997177124, 1.0004650354385376, 0.99822598695755, 0.9997280240058899, 0.9997280240058899, 134428975)
(datetime.datetime(2023, 6, 8, 0, 0), 0.999692976474762, 1.001134991645813, 0.9985820055007935, 0.9995399713516235, 0.9995399713516235, 96623143)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 0.9997310042381288, 1.001041054725647, 0.9975460171699524, 0.9999150037765504, 0.9999

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 0.07017499953508377, 0.07120499759912491, 0.06940100342035294, 0.07103300094604492, 0.07103300094604492, 156857411)
(datetime.datetime(2023, 6, 11, 0, 0), 0.06960400193929672, 0.07057700306177139, 0.06853900104761124, 0.07017699629068375, 0.07017699629068375, 157686552)
(datetime.datetime(2023, 6, 10, 0, 0), 0.0719899982213974, 0.07200399786233902, 0.06527700275182724, 0.06960099935531616, 0.06960099935531616, 395427317)
(datetime.datetime(2023, 6, 9, 0, 0), 0.07749799638986588, 0.07833199948072433, 0.0714149996638298, 0.07198400050401688, 0.07198400050401688, 303441320)
(datetime.datetime(2023, 6, 8, 0, 0), 0.07674200087785721, 0.07795900106430054, 0.07624500244855881, 0.07749500125646591, 0.07749500125646591, 160011052)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 0.07103999704122543, 0.07256700098514557, 0.0710399970

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 15.07719898223877, 15.2539644241333, 14.821678161621094, 15.149862289428711, 15.149862289428711, 99473343)
(datetime.datetime(2023, 6, 11, 0, 0), 15.065424919128418, 15.34277057647705, 14.886591911315918, 15.077999114990234, 15.077999114990234, 101667604)
(datetime.datetime(2023, 6, 10, 0, 0), 16.940832138061523, 16.9482479095459, 13.423340797424316, 15.065045356750488, 15.065045356750488, 328662649)
(datetime.datetime(2023, 6, 9, 0, 0), 16.990835189819336, 17.16238021850586, 16.82718276977539, 16.940593719482422, 16.940593719482422, 88461211)
(datetime.datetime(2023, 6, 8, 0, 0), 16.908679962158203, 17.09269142150879, 16.80794906616211, 16.990863800048828, 16.990863800048828, 87553513)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 15.149916648864746, 15.47240924835205, 15.052201271057127, 15.24944019317627, 15.249440193

In [57]:
#database_info(c, 'ADA','Date')

In [55]:
#delete_table('BTC', c)

In [54]:
#insert_new_criptos_in_db(assets)

In [53]:
#df_from_sql_query('ETH', "Date", conn, "2023-01-01", "2023-06-13")

In [ ]:
def plot(title,crypto, conn, startDate, endDate):
    
    
    df=df_from_sql_query(crypto, "Date", conn, startDate, endDate)
    
    df['Date']= pd.to_datetime(df['Date'])
    
    df=df.drop(columns=['Adj Close'])
    
#     eth_df.set_index('Date')

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.update_layout(
        title= title,
        yaxis_title='Price',

    )

    fig.write_html(title + ".html")
    fig.write_html(title + ".html")

    fig.show()

In [ ]:
plot('ETHER-USD','ETH', conn, '2022-08-01', '2022-09-14')

In [ ]:
plot('BTC-USD','BTC', conn, '2022-08-01', '2022-09-14')